In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving spark_nlp_for_healthcare.json to spark_nlp_for_healthcare.json


In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

In [3]:
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

--2021-08-12 08:21:03--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1872 (1.8K) [text/plain]
Saving to: ‘jsl_colab_setup.sh’

jsl_colab_setup.sh  100%[===================>]   1.83K  --.-KB/s    in 0s      

2021-08-12 08:21:03 (26.3 MB/s) - ‘jsl_colab_setup.sh’ saved [1872/1872]

setup Colab for PySpark 3.1.1 and Spark NLP 3.1.3
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
I

In [4]:
import json
import os
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import pandas as pd

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.1.1
Spark NLP_JSL Version : 3.1.3


In [13]:
with open('nlp_test1 (1).txt', 'r') as file:
    sample = file.read()
    
sample_df = spark.createDataFrame([[sample]]).toDF("text")

In [58]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel\
      .pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
      .setInputCols(["document"])\
      .setOutputCol("sentences")

tokenizer = Tokenizer()\
      .setInputCols(["sentences"])\
      .setOutputCol("uncheckedtoken")\

spellModel = ContextSpellCheckerModel\
    .pretrained('spellcheck_clinical', 'en', 'clinical/models')\
    .setInputCols("uncheckedtoken")\
    .setOutputCol("tokens")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentences", "tokens"])\
      .setOutputCol("embeddings")

pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models")\
      .setInputCols("sentences", "tokens")\
      .setOutputCol("pos_tags")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en")\
      .setInputCols("sentences", "pos_tags", "tokens")\
      .setOutputCol("dependencies")

clinical_ner_tagger = MedicalNerModel.pretrained("ner_jsl","en","clinical/models")\
      .setInputCols("sentences", "tokens", "embeddings")\
      .setOutputCol("ner_tags")

ner_chunker = NerConverter()\
      .setInputCols("sentences", "tokens", "ner_tags")\
      .setOutputCol("ner_chunks")

relationPairs = ["test_result-test","test-test_result",]

re_ner_chunk_filter = RENerChunksFilter()\
        .setInputCols("ner_chunks", "dependencies")\
        .setOutputCol("re_ner_chunks")\
        .setRelationPairs(relationPairs)

re_model = RelationExtractionDLModel.pretrained("redl_clinical_biobert", "en", "clinical/models")\
        .setInputCols("re_ner_chunks", "sentences")\
        .setOutputCol("relations")
 
trained_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        spellModel,
        word_embeddings,
        pos_tagger,
        clinical_ner_tagger,
        ner_chunker,
        dependency_parser,
        re_ner_chunk_filter,
        re_model])

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[OK!]
spellcheck_clinical download started this may take some time.
Approximate size to download 142.2 MB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
redl_clinical_biobert download started this may take some time.
Approximate size to download 383.4 MB
[OK!]


In [59]:
result_sample = trained_pipeline.fit(sample_df).transform(sample_df)

In [60]:
result=result_sample.selectExpr("explode(relations) as relations")\
 .select(
   "relations.metadata.chunk1",
   "relations.metadata.entity1",
   "relations.metadata.chunk2",
   "relations.metadata.entity2",
   "relations.result"
 ).toPandas()

In [61]:
for ind in result.index:
  if(result['entity1'][ind]=='Test_Result'):
    temp=result['entity1'][ind]
    result['entity1'][ind]=result['entity2'][ind]
    result['entity2'][ind]=temp
    temp=result['chunk1'][ind]
    result['chunk1'][ind]=result['chunk2'][ind]
    result['chunk2'][ind]=temp

In [62]:
result

,chunk1,entity1,chunk2,entity2,result
0,sleep study,Test,20-25%,Test_Result,TeRP
1,ejection fraction,Test,20-25%,Test_Result,O
2,cT1cN0M0,Test,116,Test_Result,TrAP
3,GS,Test,116,Test_Result,TrAP
4,cT1cN0M0,Test,3+4,Test_Result,TrAP
5,PSA,Test,13,Test_Result,O
6,PSA,Test,3+4,Test_Result,O
7,GS,Test,13,Test_Result,O
8,GS,Test,3+4,Test_Result,O
9,ejection fraction,Test,40%,Test_Result,O
